In [1]:
import os
import pickle as pkl

import numpy as np
import pandas as pd

def load_pkl(file_path) :
    
    with open(file_path, 'rb') as f:
        
        df = pkl.load(f)
        
    return df

from transformers import BertTokenizerFast

# tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-6000-32000-vocab.txt'
#                                                    , strip_accents=False
#                                                    , lowercase=False)
def save_pkl(df, file_path) :
    
    with open(file_path, 'wb') as f:
        pkl.dump(df, f)

In [2]:
import tensorflow as tf

In [12]:
dataset = tf.data.TFRecordDataset('./Q_Bert00.tfrecords')

In [24]:

tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [13]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [14]:
use_next_sentence_label = True
output_fake_labels = True
use_position_id = False

In [15]:
feature_description = {
    'input_ids': tf.io.FixedLenFeature([256], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([256], tf.int64),
    'input_mask': tf.io.FixedLenFeature([256], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([40], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([40], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([40], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}


In [25]:
dataset = dataset.map(_parse_function,
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE) # String to Example
dataset = dataset.map(_select_data_from_record,
                                                num_parallel_calls=tf.data.experimental.AUTOTUNE) # Example to InputData

TypeError: in user code:


    TypeError: tf___parse_function() takes 1 positional argument but 2 were given


In [17]:
a = next(iter(dataset))

In [20]:
a[0]['input_ids']

<tf.Tensor: shape=(256,), dtype=int64, numpy=
array([    2,  8590,  1811,  2422,  3049, 12659,    12,    15, 11106,
        5756,  6086,    61,    13,   544,     4,     4, 26570,  8144,
        3183,  5907,    12,  8842,    61,  8210,    13,  6171,    17,
        6005,    17,     4,  6064,    17, 21318,  2422,  9547, 10830,
        3132,  1501,     4,  8888,    40,  3105,  3090, 17928,  6630,
           4,     3,  8638,     4,  3129,  6172, 16354,    17,     3,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   

In [26]:
word_df = pd.DataFrame({ "학습데이터" : tokenizer_for_load.convert_ids_to_tokens(a[0]['input_ids']),
                         "세그먼트_EMB" : a[0]['segment_ids']})


In [29]:
a[0].keys()

dict_keys(['input_ids', 'input_mask', 'segment_ids', 'masked_lm_positions', 'masked_lm_ids', 'masked_lm_weights', 'next_sentence_labels'])

In [34]:
ind

29

In [35]:
a[0]['masked_lm_positions']

<tf.Tensor: shape=(40,), dtype=int64, numpy=
array([29, 40, 45, 38, 48, 14, 16, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int64)>

In [37]:
a[0]['masked_lm_ids']

<tf.Tensor: shape=(40,), dtype=int64, numpy=
array([ 6657, 13425,    17,  3049, 11660,  9288,  5649,  6008,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0], dtype=int64)>

In [36]:
a[0]['masked_lm_positions']

<tf.Tensor: shape=(40,), dtype=int64, numpy=
array([29, 40, 45, 38, 48, 14, 16, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int64)>

In [39]:
i, ind

(0, 29)

In [41]:
a[0]['masked_lm_ids'][0]

<tf.Tensor: shape=(), dtype=int64, numpy=6657>

In [45]:
a[0]['masked_lm_ids'][i].numpy()

6657

In [49]:
for i, ind in enumerate(a[0]['masked_lm_positions']) :
    ind = ind.numpy()
    word_df.loc[ind, "MASKED_LABEL"] = tokenizer_for_load.convert_ids_to_tokens([a[0]['masked_lm_ids'][i].numpy()])
    word_df.loc[ind, "MASKED_WEIGHT"] = a[0]['masked_lm_weights'][i].numpy()

In [52]:
a[0]

{'input_ids': <tf.Tensor: shape=(256,), dtype=int64, numpy=
 array([    2,  8590,  1811,  2422,  3049, 12659,    12,    15, 11106,
         5756,  6086,    61,    13,   544,     4,     4, 26570,  8144,
         3183,  5907,    12,  8842,    61,  8210,    13,  6171,    17,
         6005,    17,     4,  6064,    17, 21318,  2422,  9547, 10830,
         3132,  1501,     4,  8888,    40,  3105,  3090, 17928,  6630,
            4,     3,  8638,     4,  3129,  6172, 16354,    17,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,   

In [50]:
word_df.head(50)

,학습데이터,세그먼트_EMB,MASKED_LABEL,MASKED_WEIGHT
0,[CLS],0,[PAD],0.0
1,제임스,0,NaN,NaN
2,얼,0,NaN,NaN
3,카,0,NaN,NaN
4,##터,0,NaN,NaN
5,주니어,0,NaN,NaN
6,(,0,NaN,NaN
7,",",0,NaN,NaN
8,1924년,0,NaN,NaN
9,10월,0,NaN,NaN


In [47]:
df[idx]['NSP']

[0, 1]

In [51]:
word_df.tail(50)

,학습데이터,세그먼트_EMB,MASKED_LABEL,MASKED_WEIGHT
206,[PAD],0,NaN,NaN
207,[PAD],0,NaN,NaN
208,[PAD],0,NaN,NaN
209,[PAD],0,NaN,NaN
210,[PAD],0,NaN,NaN
211,[PAD],0,NaN,NaN
212,[PAD],0,NaN,NaN
213,[PAD],0,NaN,NaN
214,[PAD],0,NaN,NaN
215,[PAD],0,NaN,NaN


In [50]:
word_df.tail(50)

,학습데이터,라벨링,세그먼트_EMB,MASK
27,[SEP],[SEP],0,0.0
28,버,버,1,0.0
29,##기는,##기는,1,0.0
30,몇,몇,1,0.0
31,##억,##억,1,0.0
32,##베리,##베리,1,0.0
33,##나,##나,1,0.0
34,하는,하는,1,0.0
35,악마,악마,1,0.0
36,##의,##의,1,0.0


In [7]:
splited_len

13408

In [9]:
split_ = 20

In [10]:
input_files = ['./dt/wiki_ko_dict1_p.pkl', './dt/wiki_ko_dict2_p.pkl']

i = 1

for input_file in input_files :
    
    pk = load_pkl(input_file)
    keys = list(pk.keys())
    
    splited_len = len(keys) // split_
    

    for ind in range(split_) :
        
        tmp_dict = {}
        if ind == split_ - 1 :
            target_key = keys[ind*splited_len :]
        else :
            target_key = keys[ind * splited_len : (ind+1) *splited_len]
        
        for key in target_key :

            tmp_dict[key] = pk[key]
            
        save_pkl(tmp_dict, './dt/wiki_ko_dict-{}.pkl'.format(str(i).zfill(2)))
        i+=1
